In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.read_csv('/kaggle/input/customer-segmentation/Train.csv').head(5)

In [ ]:
df=pd.read_csv('/kaggle/input/customer-segmentation/Train.csv')
df=df.drop(['Var_1','Segmentation'],axis=1)
df

In [ ]:
df.info()


In [ ]:
df.describe()


To categorize better into groups, we added an another coloumn "categorized_ages"

In [ ]:
def create_age_category_column(dataframe):
    bins = [0, 18, 25, 35, 46, 100]
    labels = ['0-18', '19-25', '26-35', '36-46', '47+']
    dataframe['categorized_ages'] = pd.cut(dataframe['Age'], bins=bins, labels=labels)
    return dataframe

df = create_age_category_column(df)
df.head(5)


In [ ]:
def count_null_values(dataframe, field):
    return dataframe[field].isnull().sum()

print("Missing Values in each column:")
for column in df.columns:
    print(f"{column} : {count_null_values(df, column)}")


In [ ]:
def groupby_count(df, groupby_cols, count_col, filter_cols=None):
    selected_cols = filter_cols + groupby_cols + [count_col] if filter_cols else groupby_cols + [count_col]
    grouped_df = df[selected_cols].groupby(by=groupby_cols).count()
    return grouped_df


In [ ]:
import tabulate

result_table = groupby_count(df, ['Spending_Score', 'Ever_Married'], 'ID').reset_index()
print(tabulate.tabulate(result_table, headers='keys', tablefmt='pretty', showindex=False))



In [ ]:
pip install tabulate

In [ ]:
import tabulate

result_table = groupby_count(df, ['Profession', 'Ever_Married'], 'ID').reset_index()
print(tabulate.tabulate(result_table, headers='keys', tablefmt='pretty', showindex=False))


In [ ]:
import tabulate

result_table = groupby_count(df, ['categorized_ages', 'Ever_Married'], 'ID').reset_index()
print(tabulate.tabulate(result_table, headers='keys', tablefmt='pretty', showindex=False))


In [ ]:
def update_ever_married(df, condition, value):
    x = df.query(condition + " and Ever_Married.isna()")
    x['Ever_Married'] = value
    df.update(x)

update_ever_married(df, "Spending_Score == 'High'", 'Yes')
update_ever_married(df, "Profession == 'Healthcare'", 'No')
update_ever_married(df, "categorized_ages == '47+'", 'Yes')
update_ever_married(df, "categorized_ages == '19-25'", 'No')
update_ever_married(df, "categorized_ages == '26-35'", 'No')


In [ ]:
print(count_null_values(df,"Ever_Married"))


In [ ]:
df.query("Ever_Married.isna()")['categorized_ages'].unique()


In [ ]:
result_table = groupby_count(df.query("categorized_ages == '36-46'"), ['categorized_ages', 'Profession', 'Ever_Married'], 'ID').reset_index()

print(tabulate.tabulate(result_table, headers='keys', tablefmt='pretty', showindex=False))


In [ ]:
print(count_null_values(df,'Ever_Married'))

In [ ]:
def update_ever_married_condition(df, age_cat_condition, profession_condition):
    x = df.query(f"{age_cat_condition} and Ever_Married.isna() and {profession_condition}")
    y = df.query(f"{age_cat_condition} and Ever_Married.isna() and not {profession_condition}")

    x['Ever_Married'] = 'Yes'
    y['Ever_Married'] = 'No'

    df.update(x)
    df.update(y)

update_ever_married_condition(df, "categorized_ages=='36-46'", "Profession != 'Marketing'")

print(count_null_values(df,'Ever_Married'))


Till now, we can conclude :

All Average / High Spending_Score are Married 

People in Healthcare profession are mostly not married


47+ People are mostly married


19-25 People are mostly not married


26-35 People are mostly not married



Now, Let's look at graduated vs categorized_ages to make our clustering easier

In [ ]:
count_null_values(df,'Graduated')


In [ ]:
result_table = groupby_count(df, ['categorized_ages', 'Graduated'], 'ID').reset_index()
print(tabulate.tabulate(result_table, headers='keys', tablefmt='pretty', showindex=False))


In [ ]:
result_table = groupby_count(df, ['Profession', 'Graduated'], 'ID').reset_index()
print(tabulate.tabulate(result_table, headers='keys', tablefmt='pretty', showindex=False))


In [ ]:
print(groupby_count(df, ['Ever_Married','Graduated'], 'ID') )


In [ ]:
def update_graduated_condition(df, condition, value):
    x = df.query(f"{condition} and Graduated.isna()")
    x['Graduated'] = value
    df.update(x)

update_graduated_condition(df, "(categorized_ages=='0-18' or categorized_ages=='19-25')", 'No')
update_graduated_condition(df, "Profession=='Artist'", 'Yes')
update_graduated_condition(df, "(categorized_ages=='47+')", 'Yes')
update_graduated_condition(df, "Ever_Married=='Yes'", 'Yes')


In [ ]:
count_null_values(df,'Graduated')


In [ ]:
df=df.drop(df.query("Graduated.isna()").index)


From Graduated, we have learnt 

0-18 People Are mostly not graduated

19-25 People are mostly not graduated

47+ People are mostly graduated

Most Artist are graduated

There is more chance that married people are Graduated


In [ ]:
count_null_values(df,"Profession")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_count_by_profession(df, condition, figsize=(10, 10)):
    filtered_df = df.query(condition)
    plt.figure(figsize=figsize)
    sns.countplot(data=filtered_df, x="Profession")

plot_count_by_profession(df, "Age > 60")
plt.show()


In [ ]:
x=df.query("Age>60")
groupby_count(x, ['Spending_Score','Profession'], 'ID') 


In [ ]:
def update_profession_condition(df, age_condition, profession_condition, spending_condition, new_profession):
    x = df.query(f"{age_condition} and {profession_condition} and {spending_condition}")
    x['Profession'] = new_profession
    df.update(x)

update_profession_condition(df, "Age > 60", "Profession.isna()", "Spending_Score=='Average'", 'Artist')
update_profession_condition(df, "Age > 60", "Profession.isna()",
                             "(Spending_Score=='Low' or Spending_Score=='High')", 'Lawyer')


In [ ]:
def update_profession_for_graduates(df, graduated_value, new_profession):
    x = df.query(f"Graduated=='{graduated_value}' and Profession.isna()")
    x['Profession'] = new_profession
    df.update(x)

update_profession_for_graduates(df, 'Yes', 'Artist')


In [ ]:

result_table = groupby_count(df, ['categorized_ages', 'Profession'], 'ID').reset_index()

print(tabulate.tabulate(result_table, headers='keys', tablefmt='pretty', showindex=False))


In [ ]:
def update_profession_based_on_age(df, age_condition, new_profession):
    x = df.query(f"{age_condition} and Profession.isna()")
    x['Profession'] = new_profession
    df.update(x)

update_profession_based_on_age(df, "(categorized_ages=='0-18' or categorized_ages=='19-25')", 'Healthcare')
update_profession_based_on_age(df, "Age > 47 and Age < 60", 'Artist')


In [ ]:

result_table = groupby_count(df, ['Ever_Married', 'Profession'], 'ID').reset_index()

print(tabulate.tabulate(result_table, headers='keys', tablefmt='pretty', showindex=False))


In [ ]:
def update_profession_based_on_marriage(df, marriage_status, new_profession):
    x = df.query(f"Ever_Married=='{marriage_status}' and Profession.isna()")
    x['Profession'] = new_profession
    df.update(x)

update_profession_based_on_marriage(df, 'Yes', 'Artist')
update_profession_based_on_marriage(df, 'No', 'Healthcare')


from profession,

People that are 60+ years old are :

mostly Artist if they have an average Spending_Score

mostly Lawyers if they have a High/Low Spending_Score

Not Married People have a Healthcare profession

Married People are Artists ! (Again)

In [ ]:
count_null_values(df,"Work_Experience")


In [ ]:
percentiles_list = [i * 0.1 for i in range(10)]
result = df['Work_Experience'].describe(percentiles=percentiles_list)

print(result)


In [ ]:
def fillna_forward_fill(df, column):
    df[column] = df[column].fillna(method='pad')


fillna_forward_fill(df, 'Work_Experience')


In [ ]:
def custom_describe_with_percentiles(series, percentiles):
    return series.describe(percentiles=percentiles)

percentiles_list = [i * 0.1 for i in range(10)]
result = custom_describe_with_percentiles(df['Work_Experience'], percentiles_list)

print(result)


for working experince,padding was done to see if there will be any changes, nothing much so we can proceed!

In [ ]:
count_null_values(df,"Family_Size")


In [ ]:
def custom_describe_with_percentiles(series, percentiles):
    return series.describe(percentiles=percentiles)

percentiles_list = [i * 0.1 for i in range(10)]
result = custom_describe_with_percentiles(df['Family_Size'], percentiles_list)

print(result)


In [ ]:
def fillna_forward_fill(df, column):
    df[column] = df[column].fillna(method='pad')

fillna_forward_fill(df, 'Family_Size')


In [ ]:
def custom_describe_with_percentiles(series, percentiles):
    return series.describe(percentiles=percentiles)

family_size_description = custom_describe_with_percentiles(df['Family_Size'], [i * 0.1 for i in range(10)])
print(family_size_description)


In [ ]:
def preprocess_data(input_df):
    df = input_df.copy()

    df = df.drop(labels=['ID', 'categorized_ages'], axis=1)

    professions = list(df['Profession'].unique())
    df['Profession'] = df['Profession'].apply(lambda x: professions.index(x) + 1)

    df['Ever_Married'] = df['Ever_Married'].map({'Yes': 1, 'No': 0})
    df['Graduated'] = df['Graduated'].map({'Yes': 1, 'No': 0})
    df['Gender'] = df['Gender'].map({'Male': 1, 'Female': 0})

    spending_scores = list(df['Spending_Score'].unique())
    df['Spending_Score'] = df['Spending_Score'].apply(lambda x: spending_scores.index(x) + 1)

    return df

original_df = df  
df = preprocess_data(original_df)


In [ ]:
df.head(5)

In [ ]:
from kmodes.kprototypes import KPrototypes
df.head(2)


In [ ]:
categorical_features_idx=[0,1,3,4,5,6]
mark_array=df.values

In [ ]:
kproto = KPrototypes(n_clusters=4, verbose=2, max_iter=20).fit(mark_array, categorical=categorical_features_idx)

In [ ]:
def add_clusters_and_copy_attributes(df, original_df, mark_array, categorical_features_idx):
    clusters = kproto.predict(mark_array, categorical=categorical_features_idx)
    df['cluster'] = list(clusters)

    df[['Profession', 'Gender', 'Ever_Married', 'Graduated', 'Spending_Score', 'ID']] = original_df[
        ['Profession', 'Gender', 'Ever_Married', 'Graduated', 'Spending_Score', 'ID']
    ]

add_clusters_and_copy_attributes(df, original_df, mark_array, categorical_features_idx)


In [ ]:
def plot_cluster_professions(df, cluster_numbers, rows=2, cols=2):
    plt.figure(figsize=(cols * 10, rows * 7))

    for i, cluster_number in enumerate(cluster_numbers):
        plt.subplot(rows, cols, i + 1)

        plt.title(f"Cluster n°{cluster_number}")

        cluster_df = df.query(f"cluster == {cluster_number}")

        sns.countplot(data=cluster_df, x="Profession")

plot_cluster_professions(df, [0, 1, 2, 3])
plt.show()


In [ ]:
def plot_ever_married_pie_charts(df, cluster_numbers, rows=2, cols=2):
    plt.figure(figsize=(cols * 10, rows * 7))

    for i, cluster_number in enumerate(cluster_numbers):
        plt.subplot(rows, cols, i + 1)

        cluster_df = df.query(f"cluster == {cluster_number}")

        ever_married_counts = cluster_df['Ever_Married'].value_counts()

        plt.pie(ever_married_counts, labels=ever_married_counts.index, autopct='%1.1f%%')

        plt.title(f'Ever Married for Cluster n°{cluster_number}')

    plt.show()

plot_ever_married_pie_charts(df, [0, 1, 2, 3])


In [ ]:
def plot_cluster_graduated_pie_chart(df, cluster_numbers, rows=2, cols=2):
    plt.figure(figsize=(cols * 10, rows * 7))

    for i, cluster_number in enumerate(cluster_numbers):
        plt.subplot(rows, cols, i + 1)

        cluster_df = df.query(f"cluster == {cluster_number}")

        graduated_counts = cluster_df['Graduated'].value_counts()

        plt.pie(graduated_counts, labels=graduated_counts.index, autopct='%1.1f%%', startangle=90)

        plt.title(f'Graduated for Cluster n°{cluster_number}')

    plt.show()

plot_cluster_graduated_pie_chart(df, [0, 1, 2, 3])


In [ ]:
def plot_cluster_spending_score(df, cluster_numbers, rows=2, cols=2):
    plt.figure(figsize=(cols * 10, rows * 7))

    for i, cluster_number in enumerate(cluster_numbers):
        plt.subplot(rows, cols, i + 1)

        cluster_df = df.query(f"cluster == {cluster_number}")

        data = cluster_df['Spending_Score'].value_counts()

        plt.pie(data, labels=data.index, autopct='%1.1f%%')

        plt.title(f'Spending Score for Cluster n°{cluster_number}')

    plt.show()

plot_cluster_spending_score(df, [0, 1, 2, 3])


In [ ]:
def plot_cluster_feature(df, cluster_numbers, feature, rows=2, cols=2):
    plt.figure(figsize=(cols * 10, rows * 7))

    for i, cluster_number in enumerate(cluster_numbers):
        plt.subplot(rows, cols, i + 1)

        plt.title(f"Cluster n°{cluster_number}")

        cluster_df = df.query(f"cluster == {cluster_number}")

        sns.countplot(data=cluster_df, x=feature)

plot_cluster_feature(df, [0, 1, 2, 3], 'Family_Size')
plt.show()


In [ ]:
def calculate_cluster_means(df, grouping_column, numeric_column):
    return df[[grouping_column, numeric_column]].groupby(by=[grouping_column]).mean()

cluster_age_means = calculate_cluster_means(df, 'cluster', 'Age')
print(cluster_age_means)


Summary of Cluster Characteristics:

Cluster 0:
- Dominant Professions: Mostly in the fields of Arts, Entertainment, and Engineering.
- Marital Status: Mostly Married (66%) and some Singles (34%).
- Graduation Status: Majority Graduated (76%) and some Non-graduates (24%).
- Spending Score Distribution: Low (55%), Average (34%), High (11%).
- Family Size: Predominantly 2, with occasional instances of 1 or 3.
- Average Age: 40.
- Description: Cluster 0 represents customers in artistic, entertainment, and engineering fields with a balanced approach to spending. They are likely to be budget-conscious shoppers, willing to spend on average-priced items, and are in the prime of their careers, possibly maintaining a balance between work and personal life.

Cluster 1:
- Dominant Professions: Mainly in Healthcare, followed by Doctors and Artists.
- Marital Status: Mostly Single (82%) and a minority Married (18%).
- Graduation Status: Mostly Non-graduates (63%) with some Graduates (37%).
- Spending Score Distribution: Low (88%), Average (8%), High (4%).
- Family Size: Primarily 4, 3, or 2.
- Average Age: 26.
- Description: Cluster 1 represents customers in healthcare and artistic fields, primarily young and budget-conscious. They are likely to be family-oriented.

Cluster 2:
- Dominant Professions: Mainly in the Legal field, followed by Artists and Executives.
- Marital Status: Mostly Married (95%) with a small percentage Single (5%).
- Graduation Status: Majority Graduates (63%) and some Non-graduates (37%).
- Spending Score Distribution: Low (41%), Average (13%), High (46%).
- Family Size: Predominantly 2, with occasional instances of 1 or 3.
- Average Age: 74.
- Description: Cluster 2 represents customers in legal and artistic fields, mostly married and graduated, with a varied age range. They are likely to be budget-conscious shoppers, more likely to be single, and include a significant number of customers with a high spending score.

Cluster 3:
- Dominant Professions: Mainly in Arts, Entertainment, and Executives.
- Marital Status: Mostly Married (88%) with a minority Single (12%).
- Graduation Status: Majority Graduates (80%) and some Non-graduates (20%).
- Spending Score Distribution: Low (38%), Average (44%), High (18%).
- Family Size: Primarily 2, with occasional instances of 3 or 4.
- Average Age: 54.
- Description: Cluster 3 represents customers in artistic and executive fields, mostly married and graduated, with an average age of 54. This cluster has a higher number of customers with an average spending score.

Conclusion:
This analysis provides insights into the characteristics of each cluster. Feedback, opinions, and suggestions for improvement are welcomed.